In [1]:
from src.utils import *

In [15]:
# import wav data to keras dataset
import numpy as np
from keras.utils import audio_dataset_from_directory

train_dataset, validation_dataset = audio_dataset_from_directory(
    directory='data/train/audio/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    validation_split=0.1,
    subset='both',
    seed=123,
    shuffle=False,
    ragged=False,
    output_sequence_length=2,
    # sampling_rate=16000,

)


Found 64721 files belonging to 30 classes.
Using 58249 files for training.
Using 6472 files for validation.


In [5]:
from transformers import TFWav2Vec2ForSequenceClassification, AutoFeatureExtractor
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Load pre-trained Wav2Vec2 model and processor using AutoFeatureExtractor
model_name = "facebook/wav2vec2-base-960h"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

# Load pre-trained Wav2Vec2 model
model = TFWav2Vec2ForSequenceClassification.from_pretrained(model_name)


TFWav2Vec2ForSequenceClassification has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFWav2Vec2ForSequenceClassification: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing TFWav2Vec2ForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2ForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFWav2Vec2ForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['wav2vec2.masked_spec_embed', 'pr

In [6]:
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [13]:
def xd(x):
    return feature_extractor(x, sampling_rate = 16000)
# Preprocess data using the feature extractor
train_dataset = train_dataset.map(
    lambda x, y: (xd(x), y)
)


Tensor("args_0:0", shape=(None, None, 2, None), dtype=float32)


ValueError: in user code:

    File "C:\Users\adamm\AppData\Local\Temp\ipykernel_27568\2652659927.py", line 5, in None  *
        lambda x, y: (xd(x), y)
    File "C:\Users\adamm\AppData\Local\Temp\ipykernel_27568\2652659927.py", line 2, in xd  *
        return feature_extractor(x, sampling_rate = 16000)
    File "c:\Users\adamm\miniconda3\envs\py310\lib\site-packages\transformers\models\wav2vec2\feature_extraction_wav2vec2.py", line 199, in __call__  *
        padded_inputs = self.pad(
    File "c:\Users\adamm\miniconda3\envs\py310\lib\site-packages\transformers\feature_extraction_sequence_utils.py", line 224, in pad  *
        return BatchFeature(batch_outputs, tensor_type=return_tensors)
    File "c:\Users\adamm\miniconda3\envs\py310\lib\site-packages\transformers\feature_extraction_utils.py", line 78, in __init__  **
        self.convert_to_tensors(tensor_type=tensor_type)
    File "c:\Users\adamm\miniconda3\envs\py310\lib\site-packages\transformers\feature_extraction_utils.py", li

In [ ]:
def get_first_batch(dataset):
    for batch in dataset:
        return batch

get_first_batch(train_dataset)

(<tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(32, 30), dtype=float32, numpy=
 array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
features, labels = next(iter(train_dataset))
temp = feature_extractor(
    features.numpy(),
    return_tensors="tf",
    sampling_rate=16000,
)
temp["input_values"][0]

ValueError: Only mono-channel audio is supported for input to Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}


In [ ]:
features

<tf.Tensor: shape=(32, 2, 1), dtype=float32, numpy=
array([[[ 9.1552734e-05],
        [ 3.0517578e-05]],

       [[-3.3264160e-03],
        [-4.5471191e-03]],

       [[-2.5024414e-03],
        [-2.1057129e-03]],

       [[-2.4414062e-04],
        [-3.3569336e-04]],

       [[-3.6621094e-04],
        [-4.5776367e-04]],

       [[ 6.6528320e-03],
        [ 7.2937012e-03]],

       [[ 8.2092285e-03],
        [ 9.9792480e-03]],

       [[-9.1552734e-05],
        [-1.5258789e-04]],

       [[ 6.1035156e-05],
        [ 3.0517578e-05]],

       [[-3.0517578e-05],
        [-3.0517578e-05]],

       [[-7.1716309e-03],
        [-1.2084961e-02]],

       [[-3.8757324e-03],
        [-5.8898926e-03]],

       [[-5.2185059e-03],
        [-8.5754395e-03]],

       [[-3.0517578e-05],
        [-3.0517578e-05]],

       [[ 2.3193359e-03],
        [-5.8593750e-03]],

       [[ 1.5625000e-02],
        [ 3.0334473e-02]],

       [[ 0.0000000e+00],
        [ 3.0517578e-05]],

       [[ 7.4768066e-03],
    

In [ ]:


# Modify output layer for classification
num_classes = 10  # Adjust based on your dataset
model.config.num_labels = num_classes
model.classifier.out_proj = tf.keras.layers.Dense(num_classes, activation='softmax')

# Compile the model
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_data_processed,
                    validation_data=validation_data_processed,
                    epochs=epochs,
                    batch_size=batch_size)

# Evaluate the model
loss, accuracy = model.evaluate(validation_data_processed)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)